In [17]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [18]:
import os
import cv2
import skimage
import numpy as np
import h5py
import torch
from torch.autograd import Variable

In [19]:
from args import video_root, video_sort_lambda
from args import feature_h5_path, feature_h5_feats, feature_h5_lens
from args import max_frames, feature_size

In [20]:
from model import AppearanceEncoder, MotionEncoder

In [37]:
video_path = "videoExample/ZJL1004.mp4"

try:
    cap = cv2.VideoCapture(video_path)
except:
    print('can not open %s.' %video_path)
    pass

In [38]:
frames = []
frame_count = 0

while True:
    ret, frame = cap.read()
    if ret is False:
        break
    frame = frame[:,:,::-1]      #????
    frames.append(frame)
    frame_count += 1

In [39]:
indices = np.linspace(8, frame_count - 7, max_frames, endpoint=False, dtype=int)

In [41]:
frames = np.array(frames)
frames_list = frames[indices]
clip_list = []
for index in indices:
    clip_list.append(frames[index - 8 : index + 8])
clip_list = np.array(clip_list)


In [110]:
image = [[1],[1]]
image = np.array(image)

In [111]:
image

array([[1],
       [1]])

In [114]:
len(image.shape)

2

In [115]:
image = image[:, :, None]

In [116]:
image.shape

(2, 1, 1)

In [117]:
if len(image.shape) == 2:
    # 把单通道的灰度图复制三遍变成三通道的图片
    image = np.tile(image[:, :, None], 3)
elif len(image.shape) == 4:
    image = image[:, :, :, 0]

In [118]:
len(image.shape)

3

In [119]:
image = np.tile(image[:, :, None], 3)

In [120]:
image

array([[[[1, 1, 1]]],


       [[[1, 1, 1]]]])

In [122]:
len(image.shape)

4

In [123]:
image.shape

(2, 1, 1, 3)

In [11]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import os
import cv2
import h5py
import numpy as np
import skimage
import tensorflow as tf

video_path = "../ZJL2478.mp4"

max_frames = 60
frames = []
frame_count = 0
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    
    if ret is False:
        break
    a=frame
    frame = frame[:, :, ::-1]         #???????????????
    b=frame
    frames.append(frame)
    frame_count += 1
if frames == []:
    print("can not open %s", video_path)
    
indices = np.linspace(8, frame_count - 7, max_frames, endpoint=False, dtype=int)
frames = np.array(frames)
frame_list = frames[indices]
clip_list = []
for index in indices:
    clip_list.append(frames[index - 8:index + 8])
clip_list = np.array(clip_list)

def frame_list_resize(frame_list,target_height = 224,target_width = 224):
    frame_list_new = []
    for image in frame_list:
        if len(image.shape) == 2:
            # 把单通道的灰度图复制三遍变成三通道的图片
            image = np.tile(image[:, :, None], 3)
        elif len(image.shape) == 4:
            image = image[:, :, :, 0]

        height, width, channels = image.shape            

        if height == width:
            resized_image = cv2.resize(image, (target_height, target_width))
        elif height < width:
            #按原图像的比例缩放，剪切
            resized_image = cv2.resize(image, (int(width * target_height / height),
                                               target_width))
            cropping_length = int((resized_image.shape[1] - target_height) / 2)
            resized_image = resized_image[:,
                                          cropping_length:resized_image.shape[1] - cropping_length]
        else:
            resized_image = cv2.resize(image, (target_height,
                                               int(height * target_width / width)))
            cropping_length = int((resized_image.shape[0] - target_width) / 2)
            resized_image = resized_image[cropping_length:
                                          resized_image.shape[0] - cropping_length] 

        image = skimage.img_as_float(resized_image).astype(np.float32)
        # 根据在ILSVRC数据集上的图像的均值（RGB格式）进行白化
        image -= np.array([0.485, 0.456, 0.406])         #优化？？？？
        image /= np.array([0.229, 0.224, 0.225])
        frame_list_new.append(image)
        
    return np.array(frame_list_new)

frame_list= frame_list_resize(frame_list)
print(frame_list.shape)
#frame_list = frame_list.transpose((0, 3, 1, 2))       #!!!!!
frame_list = tf.to_float(frame_list)
#tf.keras.backend.set_session(sess)
model_a = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',
                                                            input_shape=(
                                                                224, 224, 3),
                                                            pooling='avg')
        #'avg' 表示全局平均池将应用于最后一个卷积层的输出，因此模型的输出将是2D张量

info = model_a(frame_list)
print(info.shape)

(60, 224, 224, 3)


In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import os
import cv2
import h5py
import numpy as np
import skimage
import tensorflow as tf

video_path = "../ZJL2478.mp4"

max_frames = 20
frames = []
frame_count = 0
a_feature_size = 2048
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    
    if ret is False:
        break
    a=frame
    frame = frame[:, :, ::-1]         #???????????????
    b=frame
    frames.append(frame)
    frame_count += 1
if frames == []:
    print("can not open %s", video_path)
    
indices = np.linspace(8, frame_count - 7, max_frames, endpoint=False, dtype=int)
frames = np.array(frames)
frame_list = frames[indices]
clip_list = []
for index in indices:
    clip_list.append(frames[index - 8:index + 8])
clip_list = np.array(clip_list)

def frame_list_resize(frame_list,target_height = 224,target_width = 224):
    frame_list_new = []
    for image in frame_list:
        if len(image.shape) == 2:
            # 把单通道的灰度图复制三遍变成三通道的图片
            image = np.tile(image[:, :, None], 3)
        elif len(image.shape) == 4:
            image = image[:, :, :, 0]

        height, width, channels = image.shape            

        if height == width:
            resized_image = cv2.resize(image, (target_height, target_width))
        elif height < width:
            #按原图像的比例缩放，剪切
            resized_image = cv2.resize(image, (int(width * target_height / height),
                                               target_width))
            cropping_length = int((resized_image.shape[1] - target_height) / 2)
            resized_image = resized_image[:,
                                          cropping_length:resized_image.shape[1] - cropping_length]
        else:
            resized_image = cv2.resize(image, (target_height,
                                               int(height * target_width / width)))
            cropping_length = int((resized_image.shape[0] - target_width) / 2)
            resized_image = resized_image[cropping_length:
                                          resized_image.shape[0] - cropping_length] 

        image = skimage.img_as_float(resized_image).astype(np.float32)
        # 根据在ILSVRC数据集上的图像的均值（RGB格式）进行白化
        image -= np.array([0.485, 0.456, 0.406])         #优化？？？？
        image /= np.array([0.229, 0.224, 0.225])
        frame_list_new.append(image)
        
    return np.array(frame_list_new)

frame_list= frame_list_resize(frame_list)
frame_list = frame_list.transpose(( 0, 1, 2, 3))       #!!!!!

In [3]:
def resize_frame(image, target_height=224, target_width=224):
    if len(image.shape) == 2:
        # 把单通道的灰度图复制三遍变成三通道的图片
        image = np.tile(image[:, :, None], 3)
    elif len(image.shape) == 4:
        image = image[:, :, :, 0]

    height, width, channels = image.shape
    if height == width:
        resized_image = cv2.resize(image, (target_height, target_width))
    elif height < width:
        resized_image = cv2.resize(image, (int(width * target_height / height),
                                           target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,
                                      cropping_length:resized_image.shape[1] - cropping_length]
    else:
        resized_image = cv2.resize(image, (target_height,
                                           int(height * target_width / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:
                                      resized_image.shape[0] - cropping_length]
    return cv2.resize(resized_image, (target_height, target_width))

print(clip_list.shape)

clip_list = np.array([[resize_frame(x, 112, 112)
                               for x in clip] for clip in clip_list])
clip_transpose = clip_list.transpose(0, 4, 1, 2, 3).astype(np.float32)

clip_list = tf.to_float(clip_list)  
clip_transpose = tf.to_float(clip_transpose)

print(clip_list.shape)

(60, 16, 240, 320, 3)
(60, 16, 112, 112, 3)
